In [32]:
# Write function that inputs data sets (parsing and scoring). Make code that pulls
# csv file and scores it

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""

@author: bkearney
email: bkearne5@uncc.edu

"""

### GENE INFO

import numpy as np
import pandas as pd
import re
import statistics 
import allel

# Read ensembl genome and human geneinfo csvs
gene = pd.read_table('gene2ensembl')
human = pd.read_table('Homo_sapiens.gene_info')

# Create Aliases column (delimited Synonyms)
aliases = human['Synonyms'].tolist()
aliases_sep = []
for i in range(len(aliases)):
    my_list = aliases[i].split("|")
    my_list.append(human['Symbol'].iloc[i])
    aliases_sep.append(my_list)
human['Aliases'] = aliases_sep
print(human)

/Users/bkearney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/Users/bkearney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


       #tax_id   GeneID    Symbol LocusTag  \
0         9606        1      A1BG        -   
1         9606        2       A2M        -   
2         9606        3     A2MP1        -   
3         9606        9      NAT1        -   
4         9606       10      NAT2        -   
5         9606       11      NATP        -   
6         9606       12  SERPINA3        -   
7         9606       13     AADAC        -   
8         9606       14      AAMP        -   
9         9606       15     AANAT        -   
10        9606       16     AARS1        -   
11        9606       17     AAVS1        -   
12        9606       18      ABAT        -   
13        9606       19     ABCA1        -   
14        9606       20     ABCA2        -   
15        9606       21     ABCA3        -   
16        9606       22     ABCB7        -   
17        9606       23     ABCF1        -   
18        9606       24     ABCA4        -   
19        9606       25      ABL1        -   
20        9606       26      AOC1 

In [64]:
def translateCSV(df,start_param,match_in,match_out):
    
    #csvFile - input file to be translated
    #start_param - name of column to be translated (ex: 'Gene name')
    #match_in - name of column in gene_info(human) to use to match
    #match_out - output column, not necessary just to match
    
    #Import file to pd dataframe, toggle header?
#     df = pd.read_csv(csvFile)
    in_list = human[match_in].tolist()
    start_param = df[start_param].tolist()
    
    out_listoflist = []
    
    for i in range(len(start_param)):
        param = start_param[i]
    
        match_list=[]
        for j in range(len(in_list)):
            if param in in_list[j]:
                index_match = human.iloc[j]
                match_list.append(index_match)
                
        out_list = []
        for k in range(len(match_list)):
            out_list.append(match_list[k][match_out])
#             out_list.append("MATCH")
#             print(match_list[k][match_out])
        out_listoflist.append(out_list)
    df['output']=out_listoflist
    
    return df
# ['#tax_id',
#  'GeneID',
#  'Symbol',
#  'LocusTag',
#  'Synonyms',
#  'dbXrefs',
#  'chromosome',
#  'map_location',
#  'description',
#  'type_of_gene',
#  'Symbol_from_nomenclature_authority',
#  'Full_name_from_nomenclature_authority',
#  'Nomenclature_status',
#  'Other_designations',
#  'Modification_date',
#  'Feature_type',
#  'Aliases']

In [65]:
def f_score(TP,FP,FN):
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f_dia = 2 * precision * recall / (precision+recall)
    return f_dia

In [59]:
def scoring(dataF):

    nonmatches_ind = [ind for ind, x in enumerate(dataF['output']) if len(x)==0]
    matches_ind = [ind for ind, x in enumerate(dataF['output']) if len(x)!=0]
    nonmatches = dataF.iloc[nonmatches_ind]
    matches = dataF.iloc[matches_ind]

    truePos = len(dataF)-len(nonmatches)

    falsePos = 0 # How to calculate FP?
    falseNeg = 0
    fScore = f_score(truePos,falsePos,falseNeg)
#     return fScore
    return round(100*len(nonmatches)/len(dataF),2)

In [66]:
esm = pd.read_csv('13059_2019_1621_MOESM2_ESM.csv')
esm_short = esm.loc[0:1000,:]
esm = translateCSV(esm_short,'gene_symbol','Aliases','Full_name_from_nomenclature_authority')
sample_score = scoring(esm)
print(sample_score, "% recall rate")
esm

25.57 % recall rate


/Users/bkearney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Unnamed: 0,sgrna,chr,start,strand,mismatches,pam,ref,n.mismatches,pos1,pos2,pos3,gene_id,gene_symbol,isExonic,targetedGene,output
0,1,CATCTTCTTTCACCTGAACG,chr19,58351498.0,-,none,CGG,CATCTTCTTTCACCTGAACGCGG,0,NaN,NaN,NaN,GeneID:1;GeneID:503538,A1BG;A1BG-AS1,True,A1BG,[]
1,2,CTCCGGGGAGAACTCCGGCG,chr19,58350619.0,+,none,CGG,CTCCGGGGAGAACTCCGGCGCGG,0,NaN,NaN,NaN,GeneID:1;GeneID:503538,A1BG;A1BG-AS1,True,A1BG,[]
2,3,TCTCCATGGTGCATCAGCAC,chr19,58351650.0,+,none,AGG,TCTCCATGGTGCATCAGCACAGG,0,NaN,NaN,NaN,GeneID:1;GeneID:503538,A1BG;A1BG-AS1,True,A1BG,[]
3,4,TCTCCATGGTGCATCAGCAC,chr8,90844225.0,+,"8:A>G,19:G>C",GGG,TCTCCATGATGCATCAGCAGGGG,2,9.0,20.0,NaN,GeneID:64168,NECAB1,False,A1BG,[N-terminal EF-hand calcium binding protein 1]
4,5,TGGAAGTCCACTCCACTCAG,chr19,58351575.0,+,none,GGG,TGGAAGTCCACTCCACTCAGGGG,0,NaN,NaN,NaN,GeneID:1;GeneID:503538,A1BG;A1BG-AS1,True,A1BG,[]
5,6,ACAGGAAGAATTCAGTTATG,chr10,50828279.0,-,none,GGG,ACAGGAAGAATTCAGTTATGGGG,0,NaN,NaN,NaN,GeneID:29974,A1CF,True,A1CF,[APOBEC1 complementation factor]
6,7,ACAGGAAGAATTCAGTTATG,chr18,27923613.0,-,"15:C>A,18:C>A",TGG,ACAGGAAGAATTCAGGTAGGTGG,2,16.0,19.0,NaN,NaN,NaN,False,A1CF,[]
7,8,ACAGGAAGAATTCAGTTATG,chr9,96395680.0,-,"1:A>G,19:A>C",TGG,ATAGGAAGAATTCAGTTATTTGG,2,2.0,20.0,NaN,GeneID:195828,ZNF367,False,A1CF,[zinc finger protein 367]
8,9,AGTTATGTTAGGTATACCCG,chr10,50816205.0,-,none,AGG,AGTTATGTTAGGTATACCCGAGG,0,NaN,NaN,NaN,GeneID:653308;GeneID:29974,ASAH2B;A1CF,True,A1CF,[]
9,10,CTTCATTTCCCAGCCACCAA,chr10,50816052.0,-,none,AGG,CTTCATTTCCCAGCCACCAAAGG,0,NaN,NaN,NaN,GeneID:653308;GeneID:29974,ASAH2B;A1CF,True,A1CF,[]
